In [5]:
# 컨볼루션 신경망 모델
# 0. 사용할 패키지 불러오기
from keras.datasets import reuters
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D

max_features = 15000
text_max_words = 120

# 1. 데이터셋 생성하기

# 훈련셋과 시험셋 불러오기
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)

# 훈련셋과 검증셋 분리
x_val = x_train[7000:]
x_train = x_train[:7000]

# 데이터셋 전처리 : 문장 길이 맞추기
x_train = sequence.pad_sequences(x_train, maxlen=text_max_words)
x_val = sequence.pad_sequences(x_val, maxlen=text_max_words)
x_test = sequence.pad_sequences(x_test, maxlen=text_max_words)




In [7]:
from .loss import *

In [3]:
# 2. 모델 구성하기
model = Sequential()
model.add(Embedding(max_features, 128, input_length=text_max_words))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(46, activation='softmax'))

In [6]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=text_max_words))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))


In [ ]:
from keras.layers import Input
from keras.models import Model
input_word= Input()



In [10]:
# CNN input data 묶기 (0, 1, 2), (1, 2, 3), (2, 3, 4)
def holding(input2):
    newinput=[]
    for i in range(len(input2)-2):
        holdlist=[]
        newinput.append(holdlist)
        holdlist.append(input2[i])
        holdlist.append(input2[i+1])
        holdlist.append(input2[i+2])
    return newinput
            

# cosine similarity
import math
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)


# (0, 2):1, (1, 3):2, (2, 4):3
# 이 중에 왼쪽 pair cosine similarity
def side_cosine_similarity(v1_set, v2_set):
    first=cosine_similarity(v1_set[0], v2_set[0])
    second=cosine_similarity(v1_set[2], v2_set[2])
    
    return (first+second)/2


def center_cosine_similarity(v1_set, v2_set):
    return cosine_similarity(v1_set[1], v2_set[1])


def loss_sum(data_array):
    hold=holding(data_array)
    sum=0
    for n1 in range(len(hold)):
        for n2 in range(n1, len(hold)):
            loss=side_cosine_similarity(hold[n1], hold[n2])-center_cosine_similarity(hold[n1], hold[n2])
            sum+=loss**2
    return sum

In [11]:
model.compile(loss=loss_sum, optimizer='adam', metrics=['accuracy'])


TypeError: loss_sum() takes 1 positional argument but 2 were given

In [ ]:
loss_and_metrics = model.evaluate(x_test, batch_size=64)